In [ ]:
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import re

c:\Users\mridu\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import praw
import google.generativeai as genai

GEMINI_API_KEY = "KEY"  # Replace as needed
genai.configure(api_key=GEMINI_API_KEY)

MAX_ITEMS = 10

# ========== REDDIT API SETUP ==========

reddit = praw.Reddit(
    client_id="ID",
    client_secret="SECRET",
    user_agent="reddit_persona_script by u/yourusername"
)

# ========== SCRAPE USING PRAW ==========

def get_user_content(username):
    try:
        user = reddit.redditor(username)
        comments = []
        posts = []

        for i, comment in enumerate(user.comments.new(limit=MAX_ITEMS)):
            comments.append({
                "type": "Comment",
                "index": i + 1,
                "text": comment.body,
                "source": f"https://reddit.com{comment.permalink}"
            })

        for i, submission in enumerate(user.submissions.new(limit=MAX_ITEMS)):
            posts.append({
                "type": "Post",
                "index": i + 1,
                "text": submission.title,
                "source": f"https://reddit.com{submission.permalink}"
            })

        return comments + posts

    except Exception as e:
        print(f"Error fetching user data: {e}")
        return []

# ========== PROMPT GENERATION ==========
def generate_prompt(username, items):
    content = "\n\n".join([f"{item['type']} #{item['index']}:\n{item['text']}" for item in items])
    sources = "\n".join([f"{item['type']} #{item['index']}: {item['source']}" for item in items])

    return f"""
Based on the following Reddit user's posts and comments, generate a structured **User Persona**.

Use this format:

- Name (if implied)
- Age Range
- Occupation
- Interests and Hobbies
- Writing Style
- Personality Traits
- Values
- Subreddits of Interest
- Political or Social Views

Evidence:
List the source number (from the citations below) for each trait you inferred.

=== REDDIT CONTENT ===
{content}

=== CITATIONS ===
{sources}
"""

# ========== GEMINI INTEGRATION ==========
def generate_persona_with_gemini(prompt):
    try:
        model = genai.GenerativeModel(model_name="gemini-pro")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error generating persona: {e}"

# ========== SAVE ==========
def save_to_file(username, text):
    filename = f"user_persona_{username}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"\nPersona saved to {filename}")

# ========== MAIN ==========

url = input("Enter a Reddit user profile URL: ").strip()
if not url.startswith("https://www.reddit.com/user/"):
    print("Invalid Reddit profile URL.")

    username = url.rstrip("/").split("/")[-1]
    print(f"Scraping u/{username} via Reddit API...")
    items = get_user_content(username)

    if not items:
        print("No accessible comments or posts found.")

    prompt = generate_prompt(username, items)
    print("Generating persona with Gemini...")
    persona = generate_persona_with_gemini(prompt)
    save_to_file(username, persona)


Scraping u/Hungry-Move-6603...
Generating persona with Gemini...


NotFound: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.